## Analytics & Visualization
<b> Objective: </b> Interactive dashboards and plots for insights
<br>
<b> Main Functions: </b> plot_transactions_interactive, plot_transactions_multi_dashboard, plot_purchase_distributions

In [19]:
# Analytics & Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [20]:
trans_df = pd.read_csv('data/trans.csv')

In [21]:
# ========================
# Interactive Visualization
# ========================
def plot_transactions_interactive(trans_df, dimension='age_group', freq='M', metric='transaction_count'):
    df = trans_df.copy()
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    agg_df = df.groupby([pd.Grouper(key='transaction_date', freq=freq), dimension]).agg({
        'transaction_id':'count',
        'transaction_price':'sum'
    }).reset_index().rename(columns={'transaction_id':'transaction_count','transaction_price':'total_revenue'})
    
    y_axis = 'transaction_count' if metric=='transaction_count' else 'total_revenue'
    
    fig = px.line(
        agg_df,
        x='transaction_date',
        y=y_axis,
        color=dimension,
        title=f'{y_axis.replace("_"," ").title()} Over Time by {dimension}',
        labels={'transaction_date':'Date', y_axis:y_axis.replace("_"," ").title()},
        hover_data={'transaction_date':True, y_axis:True, dimension:True}
    )
    fig.update_layout(template='plotly_white', legend_title_text=dimension)
    fig.show()


In [22]:
plot_transactions_interactive(trans_df=trans_df, dimension='age_group', freq='M', metric='transaction_count')

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_transactions_multi_dashboard(trans_df, freq='M', metric='transaction_count'):
    """
    Multi-subplot interactive dashboard with separate legends for each subplot.
    Dimensions: age_group, gender, customer_type, state
    """
    df = trans_df.copy()
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    
    # Aggregate metrics
    agg_df = df.groupby([pd.Grouper(key='transaction_date', freq=freq),
                         'age_group','gender','customer_type','state']).agg({
                             'transaction_id':'count',
                             'transaction_price':'sum'
                         }).reset_index().rename(columns={'transaction_id':'transaction_count',
                                                          'transaction_price':'total_revenue'})
    
    y_axis = metric  # metric to plot
    
    # Create subplots: 2 rows, 2 cols
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Age Group','Gender','Customer Type','State'))
    
    # --- Age Group ---
    for i, group in enumerate(agg_df['age_group'].unique()):
        temp = agg_df[agg_df['age_group']==group]
        fig.add_trace(go.Scatter(x=temp['transaction_date'], y=temp[y_axis], mode='lines', 
                                 name=group, showlegend=(i==0)), row=1, col=1)
    
    # --- Gender ---
    for i, group in enumerate(agg_df['gender'].unique()):
        temp = agg_df[agg_df['gender']==group]
        fig.add_trace(go.Scatter(x=temp['transaction_date'], y=temp[y_axis], mode='lines', 
                                 name=group, showlegend=(i==0)), row=1, col=2)
    
    # --- Customer Type ---
    for i, group in enumerate(agg_df['customer_type'].unique()):
        temp = agg_df[agg_df['customer_type']==group]
        fig.add_trace(go.Scatter(x=temp['transaction_date'], y=temp[y_axis], mode='lines', 
                                 name=group, showlegend=(i==0)), row=2, col=1)
    
    # --- State ---
    for i, group in enumerate(agg_df['state'].unique()):
        temp = agg_df[agg_df['state']==group]
        fig.add_trace(go.Scatter(x=temp['transaction_date'], y=temp[y_axis], mode='lines', 
                                 name=group, showlegend=(i==0)), row=2, col=2)
    
    # Layout: separate legend positions
    fig.update_layout(
        height=900, width=1200, title_text=f"{y_axis.replace('_',' ').title()} Over Time by Dimensions",
        template='plotly_white',
        legend=dict(
            traceorder='normal',
            font=dict(size=10),
            x=1.05,
            y=1
        )
    )
    
    fig.show()

In [24]:
# Transactions over time by all four dimensions
plot_transactions_multi_dashboard(trans_df=trans_df, freq='M', metric='transaction_count')

In [25]:
# Revenue over time
plot_transactions_multi_dashboard(trans_df=trans_df, freq='M', metric='total_revenue')